# Delaunay-Watershed for geometry reconstruction: cell aggregate example

In this notebook:
- We show how to construct a multimaterial mesh from segmentation masks of cells in an aggregate.
- We show how to easily visualize the mesh.

# 0) Install Delaunay-Watershed

We install the viewing option from Delaunay-Watershed, based on Polyscope, to visualize the mesh later.

In [ ]:
pip install "delaunay-watershed-3d[viewing]"

## 1) Compute multimaterial mesh with Delaunay Watershed

A mesh is created via a "Mesh Reconstruction Algorithm". Although many variations can be created, we provide a default Mesh Reconstruction Algorithm:

In [ ]:
from dw3d import get_default_mesh_reconstruction_algorithm

dist = 3
mesh_reconstruction_algorithm = get_default_mesh_reconstruction_algorithm(min_distance=dist, print_info=True)

This algorithm is able to reconstruct a 3D Mesh from a segmentation mask.

In [ ]:
# Read segmentation image
import skimage.io as io

image_path = "../data/Images/1.tif"
image = io.imread(image_path)

# Use algorithm to create a mesh.
points, triangles, labels = mesh_reconstruction_algorithm.construct_mesh_from_segmentation_mask(image)
print("Done.\n---")

# The multimaterial mesh is:
# points: an array of 3D points (geometry)
# triangles: an array of triangles (topology)
# labels: an array of materials on each side of a triangle.
print(f"Created mesh has {len(points)} points and {len(triangles)} triangles.")

print(f"{points=}")
print(f"{triangles=}")
print(f"{labels=}")

By default, the mesh has the same dimension as the image in pixels. The dimensions of the mesh can be set easily to match the real sample size:

In [ ]:
from dw3d import center_points_around_zero, set_points_pixel_size

known_xy_size_of_pixel = 0.1  # say, in micrometers for example
known_z_size_of_pixel = 1.5  # we can take into account anisotropy in Z-direction.

points = set_points_pixel_size(points, xy_pixel_size=known_xy_size_of_pixel, z_pixel_size=known_z_size_of_pixel)
# Now, the mesh as the real size of the sample, here it is between 0 and some size in micrometers.
# It's a simple scaling of dimensions, nothing more.

print(f"{points=}")

Other operations are available, such as centering the points of the mesh around zero:

In [ ]:
points = center_points_around_zero(points)
# Now the mesh is centered around 0
print(f"{points=}")

Alternatively, you can set a global minimum and maximum for the size of the mesh:

In [ ]:
from dw3d import set_points_global_min_max

points = set_points_global_min_max(points, global_min=-1, global_max=1)
# Now the mesh is between -1 and 1.
print(f"{points=}")

The mesh can be saved easily for future use and analysis (with foambryo for example):

In [ ]:
# The multimaterial mesh can be saved in a .vtk or .rec format. In binary or ASCII.
from dw3d import save_mesh_to_rec_mesh, save_mesh_to_vtk_mesh

save_mesh_to_rec_mesh(points, triangles, labels, "mesh_1.rec", binary_mode=False)
save_mesh_to_vtk_mesh(points, triangles, labels, "mesh_1.vtk", binary_mode=True)
print("Mesh saved.")

In general, the last constructed mesh is saved within the algorithm and the post-process can also be done with it:

In [ ]:
# Alternatively, with last constructed mesh:
mesh_reconstruction_algorithm.construct_mesh_from_segmentation_mask(image)

mesh_reconstruction_algorithm.set_pixel_size(1, 1)
mesh_reconstruction_algorithm.center_around_zero()
mesh_reconstruction_algorithm.set_global_min_max(global_min=-1, global_max=1)


mesh_reconstruction_algorithm.save_to_rec_mesh("mesh_1.rec", binary_mode=False)
mesh_reconstruction_algorithm.save_to_vtk_mesh("mesh_1.vtk", binary_mode=True)

# The last mesh (with post-process operations) can be obtained:
post_processed_points, triangles, labels = mesh_reconstruction_algorithm.last_constructed_mesh

## 2) Visualize mesh with our Polyscope viewer.

Similarly, the last mesh can be visualized, with our custom viewer:

In [ ]:
from dw3d.viewing import plot_cells_polyscope

plot_cells_polyscope(mesh_reconstruction_algorithm)

# sometimes, it's better to scatter cells to understand the geometry
# plot_cells_polyscope(mesh_reconstruction_algorithm, view="Scattered", scattering_coeff=0.5)